# Tutorial ASAC

    
 ## Active Sensing using Actor-Critic Models

This tutorial shows how to use ASAC to do active sensing. Follow link [ASAC](https://static1.squarespace.com/static/59d5ac1780bd5ef9c396eda6/t/5d4736c0f352bf000130a3c2/1564948163734/Yoon.pdf) for more details. We are using the synthetic dataset (used in Section 4.3.1) as an example.

Load Necessary Packages and Functions

In [ ]:
#%% Necessary Packages
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

#%% Function Call
# X, Y Generation
'''
X: Generated from AR_Gauss
Y: Generated from SYN1
'''
from Data_Generation_X import AR_Gauss_X1
from Data_Generation_Y import Syn_Generation_Y1

# ASAC
from ASAC import ASAC

# Predictor after selection
from Predictor_G import Predictor_G

Data Generation Parameters Assign

In [ ]:
 # Parameters
n = 10000
d = 10
t = 9
    
# Phi and W for X Generation    
phi = np.asarray([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
W = np.asarray([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
    
# X, Y Variance
sigma_X = 1
sigma_Y = 0.1       

Data Generation

In [ ]:
trainX = AR_Gauss_X1 (n, d, t, phi, sigma_X)            
testX = AR_Gauss_X1 (n, d, t, phi, sigma_X) 
        
trainY = Syn_Generation_Y1(trainX, W, sigma_Y)
testY = Syn_Generation_Y1(testX, W, sigma_Y)

Cost Value Define

In [ ]:
cost_coef = 2
cost = cost_coef * np.asarray([1,1,1,1,1,1,1,1,1,1])

Run ASAC 

Inputs: trainX, trainY, testX, cost
Outputs: selected features in training and testing set

In [ ]:
trainG_hat, testG_hat = ASAC(trainX, trainY, testX, cost) 

Run Predictive Model only with Selected Features

In [ ]:
testY_hat = Predictor_G(trainX, testX, trainY, trainG_hat, testG_hat)

Performance Measure

In [ ]:
Output = np.zeros([(2*d+1)])

Temp_TPR  = np.zeros([d,])
Temp_RMSE = 0        
            
for i in range(n):
            
    for mm in range(d):
                
        Temp_TPR[mm] = Temp_TPR[mm] + np.sum(testG_hat[i][:,mm])
                    
    Temp_RMSE = Temp_RMSE + (mean_squared_error(testY[i],testY_hat[i]))
            
    for mm in range(d):
        Output[mm]  = Temp_TPR[mm]  / float(n*t)
                
    Output[d] = np.sqrt(Temp_RMSE / float(n))  

Print Performance

In [ ]:
print('Results with Cost '+str(cost_coef) + ' :')
print('Selected Ratio: ')
print(np.round(Output[:10],3))
        
print('RMSE: ')
print(Output[10])